### Regresión Logistica
Salazar Vega Rodrigo

In [1]:
# from nltk.Wrd import tokenizer
import numpy as np
import pandas as pd
import nltk

In [95]:
path = './corpus/SMS_Spam_Corpus_big.txt'
def read_corpus(path = str()):
    """
    path : ubicación del archivo a leer
    retorna una lista de textos
    """    
    with open('./corpus/SMS_Spam_Corpus_big.txt', 'r') as f:
        text = f.read()
    
    return text

def separar_texto(text = str()):
    """
    text = una cadena de texto plano que recien fue leida por python
    Separa el texto por renglon y regresa dos vectores, de la siguiente forma, primero el valor, despues la etiqueta
    ejemplo: return valor, etiquetas
    """
    # Creamos lista separando las por salto de linea
    lista_tex = text.split('\n')

    # Creamos vectores a devolver
    valor = []
    etiquetas = []

    # Ciclo para dividir en etiqueta y valor
    for lista in lista_tex:
        valor_separado = lista.split(sep=',')

        # Obtenemos el tamaño de separar cada linea por comas, se le resta 1 por que python inicia las listas desde 0
        n = len(valor_separado)
        n = n-1
        
        # Si obtenemos 2 elementos en n, lo guardamos de esa manera sin usar slice ([:n-1])
        if n <= 1:
            texto = valor_separado[n-1]
            texto = ''.join(texto)
            eti = valor_separado[n]
        else:
            texto = valor_separado[:n-1]
            texto = ''.join(texto)
            eti = valor_separado[n]

        # Agregamos los valores obtenidos en el vector correspondiente
        valor.append(texto)
        etiquetas.append(eti)

    return valor, etiquetas

def tokenizar():
    return 1>2

In [90]:
text = read_corpus(path)


In [92]:
valor, etiqueta = separar_texto(text)

['spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham'